## Droppable data and target variable exploration

In [39]:
# Imports
import pandas as pd
import numpy as np

In [40]:
# Helper functions from helper_functions.py (couldn't import for some reason)

def set_nulls(data, cols):
    """
   @param data: dataframe
   @param cols: list of column names that have -99
   
   @return dataframe with -99 replaced with NaN
    """
    for c in cols:
        idx = np.where(data[c] == -99)[0]
        if len(idx) > 0:
            data[c].loc[idx] = np.nan
            
    return data

def map_cpt(data, column, replace, name):
    """
    @param data: dataframe
    @param column: string, column name
    @param replace: list of variables holding the values to be replaced by that particular variable name
    @ param name: string or integer of what will replace the values in replacements

    """
    for r in replace:
        idx = np.where(data[column] == r)[0]
        data[column].loc[idx] = name
    
    return data

First, we're gonna clean data. I pulled the relevant bits from the first notebook. Ideally we should eventually have a single data cleaning function we can import and run on all scripts which cleans, formulates target variables, drops columns etc once we have figured all that out

In [3]:
data = pd.read_csv('../data/monet_output.csv')
data.drop(['Unnamed: 0', 'X'], axis = 1, inplace = True)
integer_cols = data.dtypes == int
int_cols = data.columns[integer_cols]
df = set_nulls(data, int_cols)
float_flag = df.dtypes == float
float_cols = df.columns[float_flag]
df_clean = set_nulls(df, float_cols)

op1 = ['COLCT TOT ABDL W/O PRCTECT W/CONTINENT ILEOST']
op2 = ['COLCT TOT ABDL W/O PRCTECT W/ILEOST/ILEOPXTS', 'LAPS COLECTOMY TOT W/O PRCTECT W/ILEOST/ILEOPXTS']
op3 = ['COLECTOMY PARTIAL W/ANASTOMOSIS', 'LAPAROSCOPY COLECTOMY PARTIAL W/ANASTOMOSIS']
op4 = ['COLECTOMY PRTL ABDOMINAL & TRANSANAL APPROACH', 'COLECTOMY PRTL ABDOMINAL & TRANSANAL APPR']
op5 = ['COLECTOMY PRTL W/COLOPROCTOSTOMY', 'LAPS COLECTOMY PRTL W/COLOPXTSTMY LW ANAST']
op6 = ['COLECTOMY PRTL W/COLOPROCTOSTOMY & COLOSTOMY', 'LAPS COLECTMY PRTL W/COLOPXTSTMY LW ANAST W/CLST']
op7 = ['COLECTOMY PRTL W/COLOST/ILEOST & MUCOFISTULA']
op8 = ['COLECTOMY PRTL W/END COLOSTOMY & CLSR DSTL SGMT', 'COLECTOMY PRTL W/END COLOSTOMY&CLSR DSTL SGMT', 'LAPS COLECTOMY PRTL W/END CLST & CLSR DSTL SGM', 'LAPS COLECTOMY PRTL W/END CLST&CLSR DSTL SGM']
op9 = ['COLECTOMY PRTL W/RMVL TERMINAL ILEUM & ILEOCOLOS', 'COLECTOMY PRTL W/RMVL TERMINAL ILEUM&ILEOCOLOST', 'LAPS COLECTOMY PRTL W/RMVL TERMINAL ILEUM', 'COLECTOMY PRTL W/RMVL TERMINAL ILEUM & ILEOCOLOST']
op10 = ['COLECTOMY PRTL W/SKIN LEVEL CECOST/COLOSTOMY']
num_replacements = [op1, op2, op3, op4, op5, op6, op7, op8, op9, op10]
for i in range(len(num_replacements)):
    df_clean = map_cpt(df_clean, 'PRNCPTX', num_replacements[i], i+1)

MIS = ['Laparoscopic', 'Endoscopic w/ unplanned conversion to open', 'Hybrid', 'Hybrid w/ open assist', 'Laparoscopic Hand Assisted', 'Laparoscopic w/ open assist', 'Laparoscopic w/ unplanned conversion to open', 'Laparoscopic w/ unplanned conversion to Open', 'Other MIS approach', 'Robotic', 'Robotic w/ open assist', 'Robotic w/ unplanned conversion to open', 'SILS', 'SILS w/ open assist', 'SILS w/ unplanned conversion to open', 'Hybrid w/ unplanned conversion to open', 'Endoscopic w/ open assist', 'Other MIS approach w/ open assist', 'Endoscopic', 'NOTES', 'NOTES w/ open assist', 'Other MIS approach w/ unplanned conversion to open', 'NOTES w/ unplanned conversion to open']
Open = ['Open', 'Open (planned)']
options = [MIS, Open]
names = ['MIS', 'open']
for i in range(len(options)):
    df_clean = map_cpt(df_clean, 'COL_APPROACH', options[i], names[i])

#convert unknowns to NAs
nulls = np.where(df_clean.COL_APPROACH == 'Unknown')[0]
df_clean.COL_APPROACH.loc[nulls] = np.nan

/Users/kpant/.pyenv/versions/3.9.1/envs/cs/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10,17,19,23,25,27,28,29,30,32,33,36,37,38,39,40,41,42,43,50,51,83,86,89,92,94,95,97,100,103,106,110,113,116,121,124,127,130,133,136,188,190,201,212,231,232,242,244,247,249,256,262,267,273,279,280,281,282,283,284,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,305,318,320,333,334) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/kpant/.pyenv/versions/3.9.1/envs/cs/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


First thing I wanna look at is the 'ATTEND' field. Seems like most of this is NaN, but we should talk to Zoran before dropping in case we can impute a third "default" value into the NaNs, in which case this could end up being a high-impact field

In [4]:
df_clean['ATTEND'].value_counts(dropna=False, normalize=True)

NaN                           0.998655
Attending & Resident in OR    0.000725
Attending Alone               0.000620
Name: ATTEND, dtype: float64

Next, I started looking at the value counts for the first few ,no values, saw the breakdown was identical...

In [5]:
df_clean['ETOH'].value_counts(dropna=False, normalize=True)

NaN    0.999194
No     0.000806
Name: ETOH, dtype: float64

In [6]:
df_clean['CPNEUMON'].value_counts(dropna=False, normalize=True)

NaN    0.999194
No     0.000806
Name: CPNEUMON, dtype: float64

...so I decided to look at all of them at once

In [7]:
cols = ['ETOH',
'CPNEUMON',
'DNR',
'ESOVAR',
'HXMI',
'PRVPCI',
'PRVPCS',
'HXANGINA',
'HXPVD',
'RESTPAIN',
'IMPSENS',
'COMA',
'HEMI',
'HXTIA',
'CVA', 
'CVANO', 
'TUMORCNS', 
'QUAD', 
'CHEMO', 
'RADIO', 
'CNSCOMA', 
'NEURODEF', 
'OTHGRAFL', 
'PARA', 
'PREGNANCY']

In [8]:
df_clean[cols].apply(pd.Series.value_counts,dropna=False, normalize=True,)

,ETOH,CPNEUMON,DNR,ESOVAR,HXMI,PRVPCI,PRVPCS,HXANGINA,HXPVD,RESTPAIN,...,CVANO,TUMORCNS,QUAD,CHEMO,RADIO,CNSCOMA,NEURODEF,OTHGRAFL,PARA,PREGNANCY
NaN,0.999194,0.999194,0.999066,0.999194,0.999194,0.999194,0.999194,0.999194,0.999194,0.999194,...,0.999194,0.999194,0.999194,0.999194,0.999376,0.850779,0.850779,0.850779,0.999194,0.999228
No,0.000806,0.000806,0.000934,0.000806,0.000806,0.000806,0.000806,0.000806,0.000806,0.000806,...,0.000806,0.000806,0.000806,0.000806,0.000624,NaN,NaN,NaN,0.000806,0.000772
No Complication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.149221,0.149221,0.149221,NaN,NaN


In [9]:
# Need to combine no complication with no

for c in ['CNSCOMA', 'NEURODEF', 'OTHGRAFL']:
    idx = np.where(df_clean[c] == 'No Complication')[0]
    df_clean[c].loc[idx] = 'No'

/Users/kpant/.pyenv/versions/3.9.1/envs/cs/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
df_clean[cols].apply(pd.Series.value_counts,dropna=False, normalize=True,)

,ETOH,CPNEUMON,DNR,ESOVAR,HXMI,PRVPCI,PRVPCS,HXANGINA,HXPVD,RESTPAIN,...,CVANO,TUMORCNS,QUAD,CHEMO,RADIO,CNSCOMA,NEURODEF,OTHGRAFL,PARA,PREGNANCY
NaN,0.999194,0.999194,0.999066,0.999194,0.999194,0.999194,0.999194,0.999194,0.999194,0.999194,...,0.999194,0.999194,0.999194,0.999194,0.999376,0.850779,0.850779,0.850779,0.999194,0.999228
No,0.000806,0.000806,0.000934,0.000806,0.000806,0.000806,0.000806,0.000806,0.000806,0.000806,...,0.000806,0.000806,0.000806,0.000806,0.000624,0.149221,0.149221,0.149221,0.000806,0.000772


In [11]:
df_clean[cols].isnull().mean() * 100

ETOH         99.919353
CPNEUMON     99.919353
DNR          99.906558
ESOVAR       99.919353
HXMI         99.919353
PRVPCI       99.919353
PRVPCS       99.919353
HXANGINA     99.919353
HXPVD        99.919353
RESTPAIN     99.919353
IMPSENS      99.919353
COMA         99.919353
HEMI         99.919353
HXTIA        99.919353
CVA          99.919353
CVANO        99.919353
TUMORCNS     99.919353
QUAD         99.919353
CHEMO        99.919353
RADIO        99.937576
CNSCOMA      85.077914
NEURODEF     85.077914
OTHGRAFL     85.077914
PARA         99.919353
PREGNANCY    99.922842
dtype: float64

In [12]:
df_clean[cols].count()

ETOH           208
CPNEUMON       208
DNR            241
ESOVAR         208
HXMI           208
PRVPCI         208
PRVPCS         208
HXANGINA       208
HXPVD          208
RESTPAIN       208
IMPSENS        208
COMA           208
HEMI           208
HXTIA          208
CVA            208
CVANO          208
TUMORCNS       208
QUAD           208
CHEMO          208
RADIO          161
CNSCOMA      38486
NEURODEF     38486
OTHGRAFL     38486
PARA           208
PREGNANCY      199
dtype: int64

In [13]:
df[df['ETOH'].notnull()][cols]

,ETOH,CPNEUMON,DNR,ESOVAR,HXMI,PRVPCI,PRVPCS,HXANGINA,HXPVD,RESTPAIN,...,CVANO,TUMORCNS,QUAD,CHEMO,RADIO,CNSCOMA,NEURODEF,OTHGRAFL,PARA,PREGNANCY
39,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,NaN
40,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,NaN
41,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,NaN
42,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,NaN
43,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4251,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
4252,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
4253,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No
4254,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,No,No


The above shows a few things: firstly, most of the data is actually missing. Secondly, whatver data we do have is all in the same 200ish rows for most of these fields, leading me to believe that this was added from another dataset. Finally, none of these fields have any Yes values, and all of these fields are highly specific conditions (meaning that "Yes" would be the rare term and cannot be imputed). As a result, what we end up having is a bunch of fields with no data whatsoever to affect any of our models. I believe these columns should be dropped

Next let us look at the readmission related reasons with mostly null values

In [14]:
other=['UNPLANNEDREADMISSION4', 
'READMRELATED4', 
'UNPLANNEDREADMISSION5', 
'READMRELATED5']

df_clean[other].isnull().mean() * 100

UNPLANNEDREADMISSION4    99.994184
READMRELATED4            99.994184
UNPLANNEDREADMISSION5    99.998449
READMRELATED5            99.998449
dtype: float64

In [15]:
df_clean[other].apply(pd.Series.value_counts,dropna=False, normalize=True,)

,UNPLANNEDREADMISSION4,READMRELATED4,UNPLANNEDREADMISSION5,READMRELATED5
NaN,0.999942,0.999942,0.999984,0.999984
1.0,0.000058,0.000058,0.000016,0.000016


So seems like these are mostly blank because very few patients had 4-5 readmissions

In [16]:
readm = [c for c in df_clean if "READM" in c]

readm

['READMISSION1',
 'READMPODAYS1',
 'UNPLANNEDREADMISSION1',
 'READMRELATED1',
 'READMSUSPREASON1',
 'READMRELICD91',
 'READMISSION2',
 'READMPODAYS2',
 'UNPLANNEDREADMISSION2',
 'READMRELATED2',
 'READMSUSPREASON2',
 'READMRELICD92',
 'READMISSION3',
 'READMPODAYS3',
 'UNPLANNEDREADMISSION3',
 'READMRELATED3',
 'READMSUSPREASON3',
 'READMRELICD93',
 'READMISSION4',
 'READMPODAYS4',
 'UNPLANNEDREADMISSION4',
 'READMRELATED4',
 'READMSUSPREASON4',
 'READMRELICD94',
 'READMISSION5',
 'READMPODAYS5',
 'UNPLANNEDREADMISSION5',
 'READMRELATED5',
 'READMSUSPREASON5',
 'READMUNRELSUSP1',
 'READMUNRELICD91',
 'READMUNRELSUSP2',
 'READMUNRELICD92',
 'READMUNRELSUSP3',
 'READMUNRELICD93',
 'READMRELICD101',
 'READMUNRELICD101',
 'READMRELICD102',
 'READMUNRELICD102',
 'READMRELICD103',
 'READMUNRELICD103',
 'READMRELICD104',
 'READMRELICD105']

As there are only 5 related and unrelated readmission reasons, it makes sense why most of them are blank toward the end

Next, its time to consider our target variables. Per our discussion, we agreed to make them a binary flag if there was any unplanned readmission

In [41]:
unplanned = [c for c in df_clean if "UNPLANNEDREADMISSION" in c]
unplanned

['UNPLANNEDREADMISSION1',
 'UNPLANNEDREADMISSION2',
 'UNPLANNEDREADMISSION3',
 'UNPLANNEDREADMISSION4',
 'UNPLANNEDREADMISSION5']

In [42]:
df_clean['num_unplanned'] = df_clean[unplanned].sum(axis=1)

df_clean['target'] = [1 if x>0 else 0 for x in df_clean['num_unplanned']]

In [43]:
df_clean['target'].value_counts(normalize=True)

0    0.900885
1    0.099115
Name: target, dtype: float64

Worth noting that it is about a 90-10 split of 0 to 1, meaning we have unbalanced data and need to model appropriately.

Just for sanity, let us also look at total readmissions to see if they line up (should be >= unplanned readmissions)

In [32]:
total = [c for c in df_clean if "UNPLANNED" not in c and "READMISSION" in c]
total

['READMISSION1',
 'READMISSION2',
 'READMISSION3',
 'READMISSION4',
 'READMISSION5']

In [33]:
df_clean['num_total'] = df_clean[total].sum(axis=1)

df_clean['target2'] = [1 if x>0 else 0 for x in df_clean['num_total']]

In [36]:
df_clean['target2'].value_counts(normalize=True)

0    0.899412
1    0.100588
Name: target2, dtype: float64

In [37]:
df_clean['target_difference'] = df_clean['target2'] - df_clean['target']

In [38]:
df_clean['target_difference'].value_counts(normalize=True)

0    0.998527
1    0.001473
Name: target_difference, dtype: float64

In [44]:
(df_clean['num_total'] - df_clean['num_unplanned']).value_counts(normalize=True)

0.0    0.998461
1.0    0.001520
2.0    0.000019
dtype: float64

As can be seen above, total readmissions is always greater than or equal to unplanned readmissions